---

Created for [learn-investments.rice-business.org](https://learn-investments.rice-business.org)
    
By [Kerry Back](https://kerryback.com) and [Kevin Crotty](https://kevin-crotty.com)
    
Jones Graduate School of Business, Rice University

---


# EXAMPLE DATA

In [13]:
maturity = 5            # years to maturity
coupon_rate = 0.05      # coupon rate (decimal notation)
price = 90              # bond price
yld = 0.06              # yield-to-maturity (annual)
solvefor = 'price'      # 'yld': solve for yield given price
                        # 'price': solve for price given yield

# CALCULATIONS

In [15]:
import numpy as np
import numpy_financial as npf
import pandas as pd
import plotly.graph_objects as go

n = maturity*2
coupon = 100*coupon_rate / 2 

if solvefor=='yld':
    cashFlows = [-price] + [coupon]*n
    cashFlows[-1] += 100
    yld = 2*npf.irr(cashFlows) 
    # alternative using npf annuity function
    # yld = 2*npf.rate(n,coupon,-price,100)
    print(f'Yield is {yld:.2%} if price is ${price:.2f}')
elif solvefor=='price':
    cashFlows = [0] + [coupon] * n
    cashFlows[-1] += 100
    price = np.sum(cashFlows * (1+yld/2)**np.arange(0, -n-1, -1)) 
    # alternative using npf annuity function
    # price = -npf.pv(yld/2, n, coupon, 100)    
    print(f'Price is ${price:.2f} if yield is {yld:.2%}')
else:
    print('Set solvefor to either price or yld')

Price is $95.73 if yield is 6.00%


# TABLE

In [16]:
df = pd.DataFrame(
    dtype=float, index=range(1, n + 1), columns=["cf", "factor", "pv"]
)
df.index.name = "period"
df["cf"] = cashFlows[1:]
df["factor"] = 1 / (1 + yld / 2) ** np.arange(1, n + 1)
df["pv"]     = df.cf * df.factor
df["factor"] = df.factor.round(3)
df[["cf", "pv"]] = df[["cf", "pv"]].round(2)
df = df.reset_index()
df.columns = ["Period", "Cash Flow", "PV Factor @ Yield", "PV of Cash Flow"]
df

,Period,Cash Flow,PV Factor @ Yield,PV of Cash Flow
0,1,2.5,0.971,2.43
1,2,2.5,0.943,2.36
2,3,2.5,0.915,2.29
3,4,2.5,0.888,2.22
4,5,2.5,0.863,2.16
5,6,2.5,0.837,2.09
6,7,2.5,0.813,2.03
7,8,2.5,0.789,1.97
8,9,2.5,0.766,1.92
9,10,102.5,0.744,76.27


# FIGURE

In [17]:
# compute arrays of yields and prices
grid = np.arange(0.01, 2 * yld + 0.01, 0.0001)
prices = [np.sum(cashFlows[1:] / (1 + y / 2) ** np.arange(1, n + 1)) for y in grid]

# make figure
fig = go.Figure()

# price as a function of yield
string = "price = $%{y:.02f} when yield = %{x:,.2%}<extra></extra>"
trace0 = go.Scatter(
    x=grid, 
    y=prices, 
    mode="lines", 
    hovertemplate=string, 
    line=dict(color='blue')
)

# price at prevailing yield
string = "price = $%{y:.02f} when yield = %{x:,.2%}<extra></extra>"
trace1 = go.Scatter(
    x=[yld],
    y=[price],
    mode="markers",
    hovertemplate=string,
    marker=dict(size=15, color='red'),
)

# horizontal line at current price
trace2 = go.Scatter(
    x=[0] + list(grid),
    y=[price] * (1 + len(grid)),
    mode="lines",
    line=dict(dash="dot", color='red'),
    hovertemplate="price<extra></extra>",
)

fig.add_trace(trace0)
fig.add_trace(trace1)
fig.add_trace(trace2)
fig.update_layout(
    xaxis_tickformat=".0%",
    xaxis_rangemode='tozero',
    yaxis_tickformat=",.0f",
    yaxis_tickprefix='$',
    xaxis_title="Yield",
    yaxis_title="Price",
    template="plotly_white",
    showlegend=False   
)
fig.show()